## Supervised learning assignment
- Name: Daan Steur
- Studentnumber: 379943
- Supervisor: Dr. Dave Langers
- Course: DSLS Data Science 3: Deep learning
- Date: 24th of june

# Title

## Assignment


## data source


In [9]:
# import packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn import naive_bayes
import os, sys, wget, tarfile, zipfile, shutil, requests
import opendatasets as od

### Data Gathering process

In [39]:
# create output folder
def create_dir(dir_name):
    if not os.path.exists(dir_name):
        os.makedirs(dir_name)
        
create_dir('data')
    
# Data download link
download_link = "https://www.kaggle.com/datasets/muratkokludataset/rice-image-dataset/download"
name = "dlsteur"
api_key = "a49873aeba9e95f082c95a51ae6c3266"

https://www.analyticsvidhya.com/blog/2021/04/how-to-download-kaggle-datasets-using-jupyter-notebook/

od.download("https://www.kaggle.com/rashikrahmanpritom/heart-attack-analysis-prediction-dataset")


# # put download into output folder
# r = requests.get(download_link, allow_redirects=True)
# with open('data/', 'wb') as zip:
#     zip.write(r.content)
# # check if file is downloaded
# if os.path.isfile('data/rice.zip'):
#     print('File downloaded successfully')



    

# unzip file

    



PermissionError: [Errno 13] Permission denied: 'data/'

### Exploration & preprocessing

### Baseline model

### Fully-connected neural network model

### Regularized convolutional neural network model